<a href="https://colab.research.google.com/github/google-research/scenic/blob/main/scenic/projects/objectvivit/objectvivit_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ObjectViViT Demo: How can objects help action recognition?

This notebook demonstrates ObjectViViT, a method that uses object detection results to improve action recognition in videos. The approach includes:

1. **Object-guided token sampling**: Drop certain input tokens with minimal accuracy impact
2. **Object-aware attention**: Enrich features with object information to improve recognition

Paper: [How can objects help action recognition?](http://arxiv.org/abs/xxxx.xxxxx) (CVPR 2023)

## Setup and Installation

In [ ]:
# Install dependencies
!pip install -q jax[cpu]
!pip install -q flax
!pip install -q ml-collections
!pip install -q absl-py
!pip install -q clu
!pip install -q tensorflow
!pip install -q tensorflow-datasets
!pip install -q opencv-python
!pip install -q matplotlib
!pip install -q numpy

# Clone the scenic repository
!git clone https://github.com/google-research/scenic.git
%cd scenic

# Install dmvr dependency
!pip install -q git+https://github.com/deepmind/dmvr.git

In [ ]:
import sys
sys.path.append('/content/scenic')

import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import ml_collections
from typing import Any, Dict, Optional

# Import ObjectViViT modules
from scenic.projects.objectvivit import model
from scenic.projects.objectvivit import model_utils
from scenic.projects.objectvivit.configs import ssv2_B16_object

print(f"JAX version: {jax.__version__}")
print(f"JAX devices: {jax.devices()}")

## Model Configuration

Let's load the default configuration for ObjectViViT on Something-Something v2 dataset.

In [ ]:
# Get the default configuration
config = ssv2_B16_object.get_config()

print("Model configuration:")
print(f"Model name: {config.model_name}")
print(f"Image size: {config.dataset_configs.image_size}")
print(f"Number of frames: {config.dataset_configs.num_frames}")
print(f"Patch size: {config.model.patches.size}")
print(f"Hidden size: {config.model.hidden_size}")
print(f"Number of layers: {config.model.num_layers}")
print(f"Number of heads: {config.model.num_heads}")
print(f"Object attention enabled: {config.model.attention_config.object_attention}")
print(f"Object sampling enabled: {config.model.attention_config.object_sampling}")

## Model Demonstration

Let's create a simplified demonstration of the ObjectViViT model components.

In [ ]:
# Create model instance
model_cls = model.ViViTModelWithObjects

# Initialize with dummy data to understand the model structure
rng = jax.random.PRNGKey(0)
dummy_batch_size = 2
num_frames = config.dataset_configs.num_frames
image_size = config.dataset_configs.image_size

# Create dummy input data
dummy_video = jnp.ones((dummy_batch_size, num_frames, image_size, image_size, 3))
dummy_objects = jnp.ones((dummy_batch_size, num_frames, 10, 4))  # 10 objects per frame, 4 coordinates
dummy_object_features = jnp.ones((dummy_batch_size, num_frames, 10, 512))  # 512-dim features

dummy_batch = {
    'inputs': dummy_video,
    'objects': dummy_objects,
    'object_features': dummy_object_features,
    'label': jnp.ones((dummy_batch_size,), dtype=jnp.int32)
}

print(f"Input video shape: {dummy_video.shape}")
print(f"Object bounding boxes shape: {dummy_objects.shape}")
print(f"Object features shape: {dummy_object_features.shape}")

In [ ]:
# Initialize the model
vivit_model_instance = model_cls(config, dataset_meta={
    'num_classes': config.dataset_configs.num_classes,
    'input_shape': (-1, num_frames, image_size, image_size, 3),
    'num_train_examples': 168913,  # SSv2 train set size
    'num_test_examples': 27157,    # SSv2 validation set size
})

print("ObjectViViT model initialized successfully!")
print(f"Model type: {type(vivit_model_instance).__name__}")

## Key Features Explanation

### 1. Object-Guided Token Sampling

ObjectViViT uses object detection results to intelligently sample video tokens, keeping only the most relevant ones for action recognition. This reduces computational cost while maintaining accuracy.

In [ ]:
# Demonstrate object-guided sampling concept
def visualize_sampling_strategy():
    # Create a mock visualization of token sampling
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # Original tokens (all patches)
    original_tokens = np.ones((8, 14, 14))  # 8 frames, 14x14 patches
    original_tokens[:, 5:9, 5:9] = 2  # Highlight object regions
    
    # Sampled tokens (keeping object-relevant patches)
    sampled_tokens = np.zeros((8, 14, 14))
    # Keep object regions and some background
    sampled_tokens[:, 4:10, 4:10] = original_tokens[:, 4:10, 4:10]
    sampled_tokens[:, ::3, ::3] = original_tokens[:, ::3, ::3]  # Sparse background sampling
    
    # Visualize one frame
    ax1.imshow(original_tokens[0], cmap='viridis', alpha=0.8)
    ax1.set_title('Original Tokens (100%)')
    ax1.set_xlabel('Spatial Dimension')
    ax1.set_ylabel('Spatial Dimension')
    
    ax2.imshow(sampled_tokens[0], cmap='viridis', alpha=0.8)
    ax2.set_title('Object-Guided Sampling (~40% tokens)')
    ax2.set_xlabel('Spatial Dimension')
    ax2.set_ylabel('Spatial Dimension')
    
    plt.tight_layout()
    plt.show()
    
    print("Object-guided sampling keeps:")
    print("✓ All tokens around detected objects")
    print("✓ Sparse sampling of background regions")
    print("✓ Maintains ~66.2% accuracy with only 40% of tokens")

visualize_sampling_strategy()

### 2. Object-Aware Attention

The model enriches video features with object information through specialized attention mechanisms.

In [ ]:
# Demonstrate object-aware attention concept
def visualize_object_attention():
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))
    
    # Mock attention maps
    frames = ['Frame 1', 'Frame 2', 'Frame 3']
    
    for i, frame_name in enumerate(frames):
        # Standard attention (more diffuse)
        standard_attention = np.random.beta(2, 5, (14, 14))
        standard_attention = standard_attention / standard_attention.max()
        
        # Object-aware attention (focused on objects)
        object_attention = np.random.beta(2, 5, (14, 14))
        # Add object focus
        object_regions = [(5, 7), (8, 10), (6, 9)]
        for oy, ox in object_regions[:i+1]:
            object_attention[oy-1:oy+2, ox-1:ox+2] += 0.8
        object_attention = object_attention / object_attention.max()
        
        axes[0, i].imshow(standard_attention, cmap='hot', alpha=0.8)
        axes[0, i].set_title(f'{frame_name}\nStandard Attention')
        axes[0, i].axis('off')
        
        axes[1, i].imshow(object_attention, cmap='hot', alpha=0.8)
        axes[1, i].set_title(f'{frame_name}\nObject-Aware Attention')
        axes[1, i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("Object-aware attention provides:")
    print("✓ Enhanced focus on object regions")
    print("✓ Better feature representation for actions")
    print("✓ Improved accuracy: 67.4% vs 66.1% baseline")

visualize_object_attention()

## Results Summary

ObjectViViT achieves strong results on Something-Something v2 dataset:

In [ ]:
# Results visualization
import matplotlib.pyplot as plt

configs = ['Baseline\n(ViViT-B/16)', 'Object Sampling\n(40% tokens)', 'Object Attention\n(Full model)']
accuracies = [66.1, 66.2, 67.4]
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(configs, accuracies, color=colors, alpha=0.8)

# Add value labels on bars
for bar, acc in zip(bars, accuracies):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
            f'{acc}%', ha='center', va='bottom', fontweight='bold')

ax.set_ylabel('Top-1 Accuracy (%)', fontsize=12)
ax.set_title('ObjectViViT Results on Something-Something v2', fontsize=14, fontweight='bold')
ax.set_ylim(65, 68)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("Key Achievements:")
print(f"• {accuracies[2] - accuracies[0]:.1f}% improvement over baseline")
print(f"• Efficient inference with {100-40}% fewer tokens (sampling variant)")
print(f"• State-of-the-art object-aware video understanding")

## Usage Instructions

To use ObjectViViT for your own projects:

1. **Install Scenic framework**:
   ```bash
   git clone https://github.com/google-research/scenic.git
   cd scenic
   pip install -r requirements.txt
   ```

2. **Install ObjectViViT dependencies**:
   ```bash
   pip install -r scenic/projects/objectvivit/requirements.txt
   ```

3. **Download pretrained checkpoints** (VideoMAE initialization)

4. **Train the model**:
   ```bash
   python -m scenic.projects.objectvivit.main \
     --config=scenic/projects/objectvivit/configs/ssv2_B16_object.py \
     --workdir=your_workdir/
   ```

## Conclusion

ObjectViViT demonstrates how object information can significantly improve video action recognition:

- **Object-guided sampling** reduces computational cost while maintaining accuracy
- **Object-aware attention** enhances feature representation
- **Strong empirical results** on challenging Something-Something v2 dataset

This approach opens up new directions for efficient and accurate video understanding.

---

**Citation:**
```bibtex
@inproceedings{zhou2023objects,
  title={How can objects help action recognition?},
  author={Zhou, Xingyi and Arnab, Anurag and Sun, Chen and Schmid, Cordelia},
  booktitle={CVPR},
  year={2023}
}
```